This notebook is meant to be an initial setup of an AI agent in AWS Bedrock. This allows for code interpretation and passing in files, which makes it ideal for an autonomous data analyst. We use Claude 3.5 Sonnet as the foundation model. This agent will be used in the user interface of the mapping application.

Based on AWS example: https://github.com/build-on-aws/agents-for-amazon-bedrock-sample-feature-notebooks/blob/main/notebooks/preview-agent-code-interpreter.ipynb

In [ ]:
import boto3
import json
import time
import csv
import io
import os

## 1) Setup

This first code block is to be run once to initialize the agent on AWS Bedrock. This includes the appropriate IAM role and policy and instruction set for agent. We choose us-west-2 because that is the region that claude 3.5 is available to use code interpreter.

In [ ]:
sts = boto3.client('sts')
caller_identity = sts.get_caller_identity()
ACCOUNT_ID = str(caller_identity['Account'])

REGION_NAME = 'us-west-2'
FOUNDATIONAL_MODEL = 'amazon.nova-premier-v1:0'
INFERENCE_PROFILE = f"us.{FOUNDATIONAL_MODEL}"
AGENT_NAME = f'climate-risk-map-ai-agent-{FOUNDATIONAL_MODEL.replace(":", "_").replace(".", "_")}'
IAM_ROLE_NAME = f'{AGENT_NAME}-role'[0:63]
MODEL_POLICY_NAME = f'{AGENT_NAME}-model-policy'
INFERENCE_POLICY_NAME = f'{AGENT_NAME}-inference-policy'
TRUST_POLICY = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": ACCOUNT_ID
                },
                "ArnLike": {
                    "aws:SourceArn": f"arn:aws:bedrock:us-west-2:{ACCOUNT_ID}:agent/*"
                }
            }
        }
    ]
}
MODEL_POLICY = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                f"arn:aws:bedrock:us-west-2::foundation-model/{FOUNDATIONAL_MODEL}"
            ]
        }
    ]
}
INFERENCE_PROFILE_POLICY = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream",
                "bedrock:GetInferenceProfile",
                "bedrock:GetFoundationModel"
            ],
            "Resource": [
                f"arn:aws:bedrock:us-west-2:{ACCOUNT_ID}:inference-profile/{INFERENCE_PROFILE}",
                f"arn:aws:bedrock:*::foundation-model/{FOUNDATIONAL_MODEL}"
            ]
        }
    ]
}
# (Keep your existing INSTRUCTION preamble about being an AI instructor, etc.)
INSTRUCTION = """
You are an advanced AI instructor for a commerical physical climate risk management education platform, whose application visualizes asset-level vulnerability to climate hazards and indicators.
Your analysis should be educational, contextual, and insightful, aiming to provide clear and actionable understanding for users managing physical climate risks.

**Core Analysis Directives:**
* **Primary Metric:** The 'ensemble_mean' column in the provided CSV represents the primary climate risk indicator (e.g., Fire Weather Index). Always use this for risk assessment unless specified otherwise.
* **Context is Key:** When presenting findings (e.g., an `ensemble_mean` value), immediately provide context using the FWI scale. For example, 'An FWI of 25 falls into the High fire danger category.'
* **Specificity Wins:** Drill down into asset specifics. Mention `osm_subtype` (e.g., 'major power lines', 'substations'), and if available and relevant, include details like `line;voltage`, `line;name`, `plant;name`, or `plant:source`.
* **Comparative Insights:** Where possible, draw comparisons. For example, compare average risk across different `osm_subtype`s or highlight assets with significantly higher risk than others in the same `county`.
* **Geospatial Focus:** Relate findings to `county` or `city` information. Frame the impact in terms of location.
* **Narrative Power:** Structure your analysis like a brief story. Instead of just listing facts, explain their implications for the infrastructure and the region.
* **Data Scope Awareness:** The provided dataset is for a specific climate scenario (e.g., SSP5-8.5), month (e.g., August), and decade (e.g., 2030s). Briefly acknowledge this scope in your initial comprehensive analyses to set user expectations.

**"Wow" Factors & Engagement:**
* **Identify the Extreme:** Proactively identify and highlight the single asset with the highest `ensemble_mean` in the dataset, or the top 2-3 if closely clustered. Describe it in detail.
* **Surface Surprises:** If you notice any surprising concentrations of risk, or particular asset types that are disproportionately affected, call these out.
* **Actionable Framing:** While educational, subtly frame insights to imply areas of concern that might warrant further investigation or mitigation planning. (e.g., "The concentration of high-risk power lines in X County, particularly those with Y voltage, suggests a potential vulnerability focus area.")
* **Suggest Follow-ups:** After providing an analysis, ALWAYS suggest 2-3 distinct and relevant follow-up questions the user might be interested in. This drives interaction and showcases your breadth. Examples:
    * "Would you like to see a breakdown of risk by asset subtype within [County X]?"
    * "Shall I identify all assets operated by [Operator Y] and their risk profiles?"
    * "Interested in the average risk for all [substations] compared to [power lines]?"
* **Educational Snippets:** Briefly explain relevant concepts if they arise. For example, if discussing a high voltage line, you could mention, "Higher voltage lines like this 500kV one are often critical for bulk power transmission, so their exposure is particularly noteworthy."

**Climate Knowledge (Keep your existing, and ensure FWI scale is prominent):**
- The Canadian Fire Weather Index (FWI) is a numerical rating...
- FWI scale interpretation:
  * 0-5: Very low fire danger
  * 5-10: Low fire danger
  * 10-20: Moderate fire danger
  * 20-30: High fire danger
  * 30+: Extreme fire danger
- This FWI does not take into account fuel on the ground and topography...
- SSP scenarios are Shared Socioeconomic Pathways...
  * SSP5-8.5: Fossil-fueled development with very high emissions (This is the scenario in the current dataset).

**Code Execution:**
You also have access to a Python environment where you can write and execute code in real-time.
When asked to perform calculations (e.g., averages, finding max/min, counting assets, filtering), data manipulations, or detailed data lookups, always use this code execution capability to ensure accuracy and to directly query the provided CSV.
After executing code, incorporate your findings directly and smoothly into your overall analysis. If you generate a table from the code, present it clearly.

**Output Format:**
* Always respond using MARKDOWN.
* Use `##` for main section titles and `###` or `####` for sub-sections if needed (NO `#` level headings).
* Use bullet points for lists and **bold text** for emphasis on key findings, metrics, and asset names.
* Be concise in each statement, but collectively thorough in your answer. Prioritize the most impactful information first.
* If a user asks a very broad question, provide a high-level summary and then offer to dive into specifics.
"""


bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = REGION_NAME)
iam = boto3.client('iam')

print("Creating the IAM policy and role...")

# Create IAM role and attach policy

try:
    role = iam.create_role(
        RoleName=IAM_ROLE_NAME,
        AssumeRolePolicyDocument = json.dumps(TRUST_POLICY)
    )

    iam.put_role_policy(
        RoleName=IAM_ROLE_NAME,
        PolicyName = MODEL_POLICY_NAME,
        PolicyDocument = json.dumps(MODEL_POLICY)
    )

    iam.put_role_policy(
        RoleName=IAM_ROLE_NAME,
        PolicyName = INFERENCE_POLICY_NAME,
        PolicyDocument = json.dumps(INFERENCE_PROFILE_POLICY)
    )
except Exception as e:
    print(f"{str(e)}")
    role = iam.get_role(RoleName=IAM_ROLE_NAME)

roleArn = role['Role']['Arn']

# --- ADD DELAY ---
print("Waiting for IAM propagation...")
time.sleep(2) # Wait for 15 seconds
# --- END DELAY ---


print(f"IAM Role: {roleArn[:13]}{'*' * 12}{roleArn[25:]}")

print("Creating the agent...")

# Create the Bedrock Agent
response = bedrock_agent.create_agent(
    agentName=AGENT_NAME,
    foundationModel=INFERENCE_PROFILE,
    instruction=INSTRUCTION,
    agentResourceRoleArn=roleArn,
)

agentId = response['agent']['agentId']

print("Waiting for agent status of 'NOT_PREPARED'...")
# Wait for agent to reach 'NOT_PREPARED' status
agentStatus = ''
while agentStatus != 'NOT_PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

######################################### Configure code interpreter for the agent
response = bedrock_agent.create_agent_action_group(
    
    actionGroupName='CodeInterpreterAction',
    actionGroupState='ENABLED',
    agentId=agentId,
    agentVersion='DRAFT',
    parentActionGroupSignature='AMAZON.CodeInterpreter' # <-  To allow your agent to generate, 
                                                        #     run, and troubleshoot code when trying 
                                                        #     to complete a task, set this field to 
                                                        #     AMAZON.CodeInterpreter. 
                                                        #     You must leave the `description`, `apiSchema`, 
                                                        #     and `actionGroupExecutor` fields blank for 
                                                        #     this action group.
)

actionGroupId = response['agentActionGroup']['actionGroupId']

print("Waiting for action group status of 'ENABLED'...")

# Wait for action group to reach 'ENABLED' status
actionGroupStatus = ''
while actionGroupStatus != 'ENABLED':
    response = bedrock_agent.get_agent_action_group(
        agentId=agentId,
        actionGroupId=actionGroupId,
        agentVersion='DRAFT'
    )
    actionGroupStatus = response['agentActionGroup']['actionGroupState']
    print(f"Action Group status: {actionGroupStatus}")
    time.sleep(2)

print("Preparing the agent...")

# Prepare the agent for use
response = bedrock_agent.prepare_agent(
    agentId=agentId
)

print("Waiting for agent status of 'PREPARED'...")

# Wait for agent to reach 'PREPARED' status
agentStatus = ''
while agentStatus != 'PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

# Create an alias for the agent
response = bedrock_agent.create_agent_alias(
    agentAliasName='test',
    agentId=agentId
)

agentAliasId = response['agentAlias']['agentAliasId']

# Wait for agent alias to be prepared
agentAliasStatus = ''
while agentAliasStatus != 'PREPARED':
    response = bedrock_agent.get_agent_alias(
        agentId=agentId,
        agentAliasId=agentAliasId
    )
    agentAliasStatus = response['agentAlias']['agentAliasStatus']
    print(f"Agent alias status: {agentAliasStatus}")
    time.sleep(2)

print('Done.\n')

print(f"agentId: {agentId}, agentAliasId: {agentAliasId}")

In [ ]:
AGENT_NAME

## 2) Utility Functions

These functions are useful to use when interacting with the agent.

In [ ]:
BEDROCK_AGENT_RUNTIME = boto3.client(service_name = 'bedrock-agent-runtime', region_name = REGION_NAME)

In [ ]:
def read_csv_bytes(file_path):
    """Reads a CSV file and returns its content as bytes.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        bytes: The content of the CSV file as bytes, or None if an error occurs.
    """
    try:
        with open(file_path, 'rb') as file:
            csv_bytes = file.read()
            return csv_bytes
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except Exception as e:
         print(f"An error occurred: {e}")
         return None

def invoke(inputText, sessionId, file_path, showTrace=False, endSession=False):

    try:

        # Invoke the Agent - Sends a prompt for the agent to process and respond to.
        response = BEDROCK_AGENT_RUNTIME.invoke_agent(
            agentAliasId=agentAliasId,   # (string) – [REQUIRED] The alias of the agent to use.
            agentId=agentId,             # (string) – [REQUIRED] The unique identifier of the agent to use.
            sessionId=sessionId,         # (string) – [REQUIRED] The unique identifier of the session. Use the same value across requests to continue the same conversation.
            inputText=inputText,         # (string) - The prompt text to send the agent.
            endSession=endSession,       # (boolean) – Specifies whether to end the session with the agent or not.
            enableTrace=True,            # (boolean) – Specifies whether to turn on the trace or not to track the agent's reasoning process.
            sessionState = {
                "files": [
                    {
                        "name": "test_data.json",
                        "source": {
                            "byteContent": {
                                "data": read_csv_bytes(file_path=file_path),
                                "mediaType": "text/csv"
                            },
                            "sourceType": "BYTE_CONTENT"
                       },
                        "useCase": "CODE_INTERPRETER"
                    }
                ]
             }
        )

        # The response of this operation contains an EventStream member. 
        event_stream = response["completion"]

        # When iterated the EventStream will yield events.
        for event in event_stream:

            # chunk contains a part of an agent response
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    text = chunk['bytes'].decode('utf-8')
                    print(f"Chunk: {text}")
                else:
                    print("Chunk doesn't contain 'bytes'")

            # files contains intermediate response for code interpreter if any files have been generated.
            if 'files' in event:
                files = event['files']['files']
                for file in files:
                    name = file['name']
                    type = file['type']
                    bytes_data = file['bytes']
                    
                    # It the file is a PNG image then we can display it...
                    if type == 'image/png':
                        # Display PNG image using Matplotlib
                        img = plt.imread(io.BytesIO(bytes_data))
                        plt.figure(figsize=(10, 10))
                        plt.imshow(img)
                        plt.axis('off')
                        plt.title(name)
                        plt.show()
                        plt.close()
                        
                    # If the file is NOT a PNG then we save it to disk...
                    else:
                        # Save other file types to local disk
                        with open(name, 'wb') as f:
                            f.write(bytes_data)
                        print(f"File '{name}' saved to disk.")

    except Exception as e:
        print(f"Error: {e}")

In [ ]:
BEDROCK_AGENT_RUNTIME = boto3.client(service_name = 'bedrock-agent-runtime', region_name = "us-west-2")

In [ ]:
BEDROCK_AGENT_RUNTIME.list_sessions(maxResults=25)